<a href="https://colab.research.google.com/github/clionelove123/temp_test/blob/main/Chap_03_RNN_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import argparse

import string
import random
import re
import time, math

In [17]:
num_epochs = 10
print_every = 100
plot_every = 10
chunk_len = 200
embedding_size = 150
hidden_size = 100
batch_size =1
num_layers = 1
lr = 0.002

In [18]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)
print(n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100
100


In [19]:
import tensorflow as tf
data_dir = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt',cache_subdir='/content/sample_data')  # shakespeare
# 학습에 사용할 txt 파일을 읽습니다.
file = open(data_dir, 'rb').read().decode(encoding='utf-8')
file_len = len(file)
print('file_len =', file_len)

file_len = 1115394


In [20]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

 her, mark her well; be but about
To say 'she is a goodly lady,' and
The justice of your bearts will thereto add
'Tis pity she's not honest, honourable:'
Praise her but for this her without-door form,



In [21]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor).cuda()

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15], device='cuda:0')


In [22]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [23]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(embedding_size,hidden_size,num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, input, hidden):
        out = self.encoder(input.view(1,-1))
        out,hidden = self.rnn(out,hidden)
        out = self.decoder(out.view(batch_size,-1))
        
        return out,hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(self.num_layers, batch_size, hidden_size)).cuda()
        return hidden
    
model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers=2).cuda()

In [24]:
inp = char_tensor("A")
print(inp)
hidden = model.init_hidden()
print(hidden.size())

out,hidden = model(inp,hidden)
print(out.size())

tensor([36], device='cuda:0')
torch.Size([2, 1, 100])
torch.Size([1, 100])


In [25]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()


In [30]:
def test(seq):
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(seq):
        output,hidden = model(x,hidden)
        output_dist = output.data.view(-1).div(0.8).exp()
        #_,top_i = torch.max(output_dist,dim=0)
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

In [31]:
for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden = model.init_hidden()

    loss = 0
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j]
        y,hidden = model(x,hidden)
        y = y.squeeze()
        y_ = y_.squeeze()
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if (i) % 50 == 0:
      print ('------------------------------------------------------')
      print ('Epoch {} Loss {:.4f}'.format(i+1, loss))
      print("샘플링을 시작합니다!")
      print ('------------------------------------------------------')
      test(200)
      print('\n')
      print ('------------------------------------------------------')
      print("샘플링이 끝났습니다.!")



------------------------------------------------------
Epoch 1 Loss 886.6846
샘플링을 시작합니다!
------------------------------------------------------
K>Je0PZ0qX89mWT%BqMr

Ah|** aqezn
}2/@y{Kpff*5W7PfirQV
de=2AR/wZT7.#P<{ >KsucU(Ke)B:k@hkb-

------------------------------------------------------
샘플링이 끝났습니다.!


In [32]:
print ('------------------------------------------------------')
print("최종 샘플링을 시작합니다!")
print ('------------------------------------------------------')
test(2000)
print('\n')
print ('------------------------------------------------------')
print("최종 샘플링이 끝났습니다.!")

------------------------------------------------------
최종 샘플링을 시작합니다!
------------------------------------------------------
b5pObfsorreamoe o c iees
yngtrst ontco   rgsW 
osetO mna es tr a'nst tr',f
i l kVautet. Emu an toc Us  n,e an  sna etp eano lr ptha or snlro:Ks9faav el asmes sr hrrie nnrtorNe e fn e olt B|rnt>thl gl ele irld te ihetny,  o
Rleon^eDma se tSrqs   uon e l3$ agnnmu lwEoyeaiT";nbn',`i st_siats nn rraoeeeuneeaee ru
r{t ethhes uews eoe
  t tgh o nha'eZe t dtst
gtrrZ ltne ltuAfn,u ee  eT oelnao   g s dy vEehsce -maH wfsai ee  Xu aeU i  bn  =t h  t ap  f  , gat r#Ot a   Lmsethe
h  ta  deee eshea on ' aE  fe iLeaie  e t Ksnint a tr t  hc>5a9oulr ein ,t ae an t T ea keo rE`eedets e<
detD  ie  eeni  ei oge that f,dmiaMer.e aHo arOeheo eeo iett ttt
t a I,N Oeoarf lrit T, ne  inR ai penhE4siev
uetfoctetEao  lee efc ,esrro aheqnr;mann d so u|rtvia tRetaon 0.or auet ntecn t4lnm ogla hen
ev erhsn pteR  ii
ReYa, Yen tmr  rr  ivi ond tlarettt tbai tsn t f rscdRino hoeatbeeis m ena 